# Vaje 11: Manjkajoče vrednosti in neenakomerne porazdelitve ciljne spremenljivke

## Naloga 1: Manjkajoče vrednosti

V praksi se pogosto spopademo s podatki, ki vsebujejo manjkajoče vrednosti. V tej nalogi si bomo pogledali nekaj pristopov za spopadanje s takšnimi podatki, kot so:
- uporaba napovednih modelov, ki se lahko spopade z njimi (npr. drevesa)
- odstranitev problematičnih stolpcev
- nadomestitev manjkajočih vrednosti v danem stolpcu s povprečjem znanih
- napoved manjkajočih vrednosti v danem stolpcu s pomočjo modela

Vse metode za delo z manjkajočimi vrednostmi vhodnih spremenljivk so primerne tudi za delo s podatki, ki jim manjkajo nekatere ciljne vrednosti, zato bodo nekatere funkcije sposobne odpraviti tudi manjkajoče vrednosti v ciljni spremenljivki (ali pa bi bile za to potrebne le majhne spremembe). Paziti je treba le (če to delamo pred razdelitvijo na učno in testno množico), da vrednosti ciljne spremenljivke iz testne ne uporabljamo.

Na današnjih vajah bomo manjkajoče vrednosti označili z NA. Za zaznavanje NA-jev je koristna funkcija
*is.na* (ter sorodna *anyNA*).


Sestavimo najprej podatkovno množico in jo razdelimo na učno in testno množico

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

p = 0.4
X = np.random.random((100, 10))
y = X * np.arange(11, 21) + X**2 * np.arange(20, 10, -1)
for i in range(100):
    if np.random.random() < p:
        X[i, np.random.randint(5, size=2)*2] = np.nan
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

1.a: Najprej uporabimo odločitvena drevesa, ki se znajo brez dodatnih izboljšav spopasti z manjkajočimi podatki. Premisli zakaj je temu tako in preveri točnost odločitvenega drevesa na danih podatkih.

1.b: Drugi pristop je, da odstanimo stolpce oziroma vrstice, ki vsebujejo manjkajoče vrednosti. Preveri točnost modela, ki uporablja podatke, ki jim odstanimo stolpce z manjkajočimi vrednostmi oziroma vrstice z manjkajočimi vrednostmi. Pomagaš si lahko s funkcijama `numpy.isnan` in `numpy.any`. Opaziš kakšen problem pri tem pristopu?

Odstranjevanje značilk ali primerov je redko dobra izbira. Preprosta in kar dobra alternativa je manjkajočo vrednost zapolniti s povprečjem (aritmetičnim ali mediano) v primeru numeričnih spremenljivk, oziroma z najpogostejšo vrednostjo v primeru kategoričnih spremenljivk.

1.c: Preveri točnost modela, ko manjkajoče vrednosti nadomestiš z aritmetičnim povprečjem in mediano. V praksi je mediana uporabna predvsem, ko imamo "outlierje", ki bi zelo pokvarili povprečje.

Najzahtevnejši pristop za obravnavo manjkajočih vrednosti je napovedovanje s pomočjo modela - za to se pogosto uporablja metoda najbližjih sosedov. Takšno imputacijo nam olajša sklearn-ov model KNNImputer. 
POZOR! Pri prejšnjih metodah ni bilo zares pomembno, pri imputaciji z modelom pa je: paziti moramo, da predprocesiranje treniramo na učni množici, njegove vrednosti pa potem uporabimo za predprocesiranje testne množice. 

1.d: Z uporabo objekta KNNImputer dopolni manjkajoče vrednosti v podatkih in preveri točnost odločitvenega drevesa na testni množici.

## Naloga 2: Neenakomerne porazdelitve ciljne spremenljivke

Kadar je porazdelitev ciljne spremenljivke močno neenakomerna, lahko pri učenju modelov nastopijo težave. Ker bolj točne napovedi za pogosto opazovane vrednosti manjšajo napako, postanejo napovedi pogostih vrednosti ciljne spremenljivke bolj pomembne, in predsodek modela se poveča. Na primer, zelo majhen delež pozitivnih primerov pri dvojiški klasifikaciji lahko vodi do modela, ki vedno napove negativni razred. 

Eden od načinov za spopadanje z neenakomernimi porazdelitvami je previdna ročna konstrukcija funkcije napake, ki da večji poudarek manj zastopanim vrednostim. 

Drug popularen pristop za reševanje teh problem je uteženo vzorčenje podatkov. Ta pristop si bomo pogledali na tej vaji.

Uporabili bomo podatkovno množico, ki vsebuje le okoli 5% pozitivnih primerov.

In [20]:
data = np.load("../Podatki/vaje11.npy", allow_pickle=True)
X = data[:, :-1]
y = data[:, -1].astype(bool)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, stratify=y)

X_train[X_train == "?"] = np.nan
X_test[X_test == "?"] = np.nan
X_train = np.array(X_train, dtype=np.float64)
X_test = np.array(X_test, dtype=np.float64)

2.a: Podatki vsebujejo manjkajoče vrednosti. Z eno od zgornjih metod te vrednosti dopolni.

Najpreprostejši (in pogosto najboljši) metodi za uteženo vzorčenje sta:

- podvzorčenje (undersampling ali downsampling): vzamemo vse primere manjšinskega razreda ter zgolj delež primerov večinskega razreda, tako da porazdelitev uravnovesimo

- nadvzročenje (oversampling ali upsampling): podatkom dodamo neko število kopij primerov majšinskega razreda, tako da porazdelitev izenačimo

Za uteževa

In [ ]:
!pip install imbalanced-learn

2.b: S pomočjo knjižnjice imabalanced-learn sestavi množico z nadvzorčenjem in podvzorčenjem. Preveri confusion matrix za model odločitvenega drevesa naučenega na navadni, nadvzorčeni in podvzorčeni učni množici.

<details>
  <summary>Namig:</summary>

Za nadvzorčenje lahko uporabiš razred [imblearn.over_sampling.RandomOverSampler](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.RandomOverSampler.html), za podvzorčenje pa razred [imblearn.under_sampling.RandomUnderSampler](https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.RandomUnderSampler.html)
   
</details>

2.c: Množico nadvzorči s pomočjo algoritma SMOTE, ki ga najdeš v paketu over_sampling knjižnjice imbalanced-learn in preveri točnost odločitvenega drevesa naučenega na tej množici na testni množici

Pomembno: Vzorčenje (posebaj nadvzorčenje) moramo delati na učni množici (pri prečnem preverjanju v vsakem fold-u posebaj). V nasprotnem primeru lahko dodamo primere, ki se pojavijo v učni množici tudi v testno množici in tako umetno izboljšamo točnost